In [117]:
import os
import collections
from collections import Counter
from pathlib import Path
import operator
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from rdkit.Chem import (
    ChemicalFeatures,
    rdDistGeom,
    Draw,
    rdMolTransforms,
    AllChem,
    Lipinski
)
from rdkit import RDConfig, Chem, Geometry, DistanceGeometry
from rdkit.Chem.Draw import DrawingOptions, IPythonConsole, rdMolDraw2D
import rdkit.Chem.AllChem as AllChem

from rdkit.Chem.Pharm3D import Pharmacophore, EmbedLib
from rdkit.Numerics import rdAlignment
import nglview as nv

from zipfile import ZipFile
from tempfile import TemporaryDirectory


from rdkit.Chem import PandasTools
from chembl_webresource_client.new_client import new_client
from tqdm.auto import tqdm



IPythonConsole.drawOptions.addAtomIndices = False
IPythonConsole.molSize = 800, 800
IPythonConsole.ipython_useSVG=True  #< set this to False if you want PNGs instead of SVGs

In [108]:
# SMILES format for ligand
smiles_code = "CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C)NC4=NC=CC(=N4)C5=CN=CC=C5"

In [109]:
# Add hydrogens to the skeleton
molecule = Chem.AddHs(Chem.MolFromSmiles(smiles_code))

In [134]:
# We need to create an embedded molecule instance in order to use nglview
AllChem.EmbedMolecule(molecule)
view = nv.show_rdkit(molecule)
view

NGLWidget()

In [27]:
# Path to the RDkit chemical features database
fdefName = os.path.join(RDConfig.RDDataDir,'BaseFeatures.fdef')

In [113]:
# Instantiate a feature factory for getting chemical properties
feature_factory = AllChem.BuildFeatureFactory(str(Path(RDConfig.RDDataDir) / "BaseFeatures.fdef"))

In [120]:
# What are the chemical features available?
list(feature_factory.GetFeatureDefs().keys())

['Donor.SingleAtomDonor',
 'Acceptor.SingleAtomAcceptor',
 'NegIonizable.AcidicGroup',
 'PosIonizable.BasicGroup',
 'PosIonizable.PosN',
 'PosIonizable.Imidazole',
 'PosIonizable.Guanidine',
 'ZnBinder.ZnBinder1',
 'ZnBinder.ZnBinder2',
 'ZnBinder.ZnBinder3',
 'ZnBinder.ZnBinder4',
 'ZnBinder.ZnBinder5',
 'ZnBinder.ZnBinder6',
 'Aromatic.Arom4',
 'Aromatic.Arom5',
 'Aromatic.Arom6',
 'Aromatic.Arom7',
 'Aromatic.Arom8',
 'Hydrophobe.ThreeWayAttach',
 'Hydrophobe.ChainTwoWayAttach',
 'LumpedHydrophobe.Nitro2',
 'LumpedHydrophobe.RH6_6',
 'LumpedHydrophobe.RH5_5',
 'LumpedHydrophobe.RH4_4',
 'LumpedHydrophobe.RH3_3',
 'LumpedHydrophobe.tButyl',
 'LumpedHydrophobe.iPropyl']

In [123]:
# How many chemical properties are present for imatinib?
features = feature_factory.GetFeaturesForMol(molecule)
print(f"Number of features found: {len(features)}")

Number of features found: 31


In [124]:
# What are the classes and counts for those chemical properties?
feature_frequency = collections.Counter([feature.GetFamily() for feature in features])
feature_frequency

Counter({'Donor': 4,
         'Acceptor': 4,
         'PosIonizable': 2,
         'Aromatic': 4,
         'Hydrophobe': 15,
         'LumpedHydrophobe': 2})

In [125]:
# Sanity check, imatinib should have 68 atoms after adding hydrogens
molecule.GetNumAtoms()

68

In [126]:
# What are the available resources we can get via the chembl webservices API?
available_resources = [resource for resource in dir(new_client) if not resource.startswith('_')]
print(available_resources)

['activity', 'activity_supplementary_data_by_activity', 'assay', 'assay_class', 'atc_class', 'binding_site', 'biotherapeutic', 'cell_line', 'chembl_id_lookup', 'compound_record', 'compound_structural_alert', 'description', 'document', 'document_similarity', 'drug', 'drug_indication', 'drug_warning', 'go_slim', 'image', 'mechanism', 'metabolism', 'molecule', 'molecule_form', 'official', 'organism', 'protein_class', 'similarity', 'source', 'substructure', 'target', 'target_component', 'target_relation', 'tissue', 'xref_source']


In [135]:
# The ChEMBL ID for imatinib is CHEMBL941, let's get the information for this molecule
imatinib_properties = new_client.molecule.get('CHEMBL941')
imatinib_properties_df = pd.DataFrame(imatinib_properties.items(), columns=['attribute', 'value'])
imatinib_properties_df

,attribute,value
0,atc_classifications,[L01EA01]
1,availability_type,1
2,biotherapeutic,None
3,black_box_warning,0
4,chebi_par_id,45783
5,chirality,2
6,cross_references,"[{'xref_id': 'imatinib%20mesylate', 'xref_name..."
7,dosed_ingredient,False
8,first_approval,2001
9,first_in_class,0


In [133]:
# Molecular properties for imatinib from ChEMBL
pd.DataFrame(imatinib_properties['molecule_properties'].items(), columns=['attribute', 'value'])

,attribute,value
0,alogp,4.59
1,aromatic_rings,4
2,cx_logd,3.80
3,cx_logp,4.38
4,cx_most_apka,12.69
5,cx_most_bpka,7.84
6,full_molformula,C29H31N7O
7,full_mwt,493.62
8,hba,7
9,hba_lipinski,8


In [119]:
# Let's see what RDkit gives us for pharacophore information and compare it to ChEMBL info
Chem.Lipinski.NumHAcceptors(molecule)

7